In [169]:
import pandas as pd
import numpy as np

In [170]:
df1 = pd.read_csv('AutoSleep-20200124-to-20231109.csv')
df2 = pd.read_csv('AutoSleep-20231018-to-20240415.csv')

# Convert 'fromDate' to datetime
# df1['fromDate'] = pd.to_datetime(df1['fromDate'])
# df2['fromDate'] = pd.to_datetime(df2['fromDate'])

# Filter out rows from df2 that are already in df1
df2 = df2[~df2['fromDate'].isin(df1['fromDate'])]

# Concatenate the DataFrames
df = pd.concat([df1, df2])

In [171]:
df = df.drop(['fellAsleepIn', 'SpO2Avg', 'SpO2Min', 'SpO2Max', 'respAvg', 'respMin', 'respMax',
              'tags', 'notes'], axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1296 entries, 0 to 140
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ISO8601         1296 non-null   object 
 1   fromDate        1296 non-null   object 
 2   toDate          1296 non-null   object 
 3   bedtime         1296 non-null   object 
 4   waketime        1296 non-null   object 
 5   inBed           1296 non-null   object 
 6   awake           1296 non-null   object 
 7   sessions        1296 non-null   int64  
 8   asleep          1296 non-null   object 
 9   asleepAvg7      1296 non-null   object 
 10  efficiency      1296 non-null   float64
 11  efficiencyAvg7  1296 non-null   float64
 12  quality         1295 non-null   object 
 13  qualityAvg7     1295 non-null   object 
 14  deep            1295 non-null   object 
 15  deepAvg7        1295 non-null   object 
 16  sleepBPM        1295 non-null   float64
 17  sleepBPMAvg7    1295 non-null   fl

In [172]:
df['ISO8601'] = pd.to_datetime(df['ISO8601'])
df['fromDate'] = pd.to_datetime(df['fromDate'], format='%A, %b %d, %Y')
df['toDate'] = pd.to_datetime(df['toDate'], format='%A, %b %d, %Y')
df['bedtime'] = pd.to_datetime(df['bedtime']).dt.time
df['waketime'] = pd.to_datetime(df['waketime']).dt.time

df['inBed'] = pd.to_datetime(df['inBed'], format='%H:%M:%S').dt.time
df['awake'] = pd.to_datetime(df['awake'], format='%H:%M:%S').dt.time
df['asleep'] = pd.to_datetime(df['asleep'], format='%H:%M:%S').dt.time
df['asleepAvg7'] = pd.to_datetime(df['asleepAvg7'], format='%H:%M:%S').dt.time


In [173]:
def med(col):
    df[col] = df[col].apply(lambda x: x.hour * 60 + x.minute)
    median_asleep = df[col].median()
    median_asleep = f"{int(median_asleep / 60)}:{int(median_asleep % 60)}"
    return print(median_asleep)

def avg(col):
    df[col] = df[col].apply(lambda x: x.hour * 60 + x.minute)
    avg_asleep = df[col].mode()
    avg_asleep = f"{int(avg_asleep / 60)}:{int(avg_asleep % 60)}"
    return print(avg_asleep)

In [174]:
med('asleep')
med('inBed')
med('awake')

7:17
7:59
0:34


In [176]:
avg('awake')

AttributeError: 'int' object has no attribute 'hour'

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1296 entries, 0 to 140
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ISO8601         1296 non-null   object        
 1   fromDate        1296 non-null   datetime64[ns]
 2   toDate          1296 non-null   datetime64[ns]
 3   bedtime         1296 non-null   object        
 4   waketime        1296 non-null   object        
 5   inBed           1296 non-null   int64         
 6   awake           1296 non-null   int64         
 7   sessions        1296 non-null   int64         
 8   asleep          1296 non-null   int64         
 9   asleepAvg7      1296 non-null   object        
 10  efficiency      1296 non-null   float64       
 11  efficiencyAvg7  1296 non-null   float64       
 12  quality         1295 non-null   object        
 13  qualityAvg7     1295 non-null   object        
 14  deep            1295 non-null   object        
 15  deepA

In [177]:
git remote add origin 'git@github.com:palmerac/sleep.git'

git push -u origin master

SyntaxError: invalid syntax (75238944.py, line 1)